In [ ]:
from sklearnex import patch_sklearn #Improves sklearn alghoritms performance
patch_sklearn()
import sklearn
print('scikit-learn version\n', sklearn.__version__)
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
import pickle
import numpy as np
import pandas as pd

In [ ]:
#Set seed
seed=41

Classification Level -> Kingdom

In [ ]:
classif_level = 0

# Load Data

In [ ]:
feature_names = pickle.load(open('Dataset/feature_names.pkl', 'rb'))
X_train = pickle.load(open(f'Dataset/X_train.pkl', 'rb'))
X_train_scal = pickle.load(open(f'Dataset/X_train_scal.pkl', 'rb'))
y_train = pickle.load(open(f'Dataset/y_train.pkl', 'rb'))[:, classif_level]

## Train and tune classifiers

### Random Forest

In [ ]:
for f_sel in ['all', 'sel']:
    if f_sel == 'all':
        X_train_ = X_train
        grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
            'criterion' : ['gini', 'entropy'],
            'max_features' : [1, 2, 3, 5, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 133]}
    else:
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Kingdom']['Chemical entities']['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train[:, f_index]
        grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
            'criterion' : ['gini', 'entropy'],
            'max_features' : range(1, len(sel_f)+1)}
    
    
    estimator = RandomForestClassifier(random_state=seed, n_jobs=-1)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', 
                      cv=3, verbose=3, error_score='raise', return_train_score=True, n_jobs=-1)
    gs.fit(X_train_, y_train)
    pickle.dump(gs, open(f'Models/Kingdom/RF_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/RF_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### KNN

In [ ]:
grid = grid = {'n_neighbors': range(1, 31), 'weights' : ['uniform', 'distance'],
               'metric':['manhattan', 'euclidean', 'cosine']}

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    if f_sel == 'sel':
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Kingdom']['Chemical entities']['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train_scal[:, f_index]
    else:
        X_train_ = X_train_scal
    estimator = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3,
                      error_score='raise', return_train_score=True, n_jobs=10, pre_dispatch=15)
    gs.fit(X_train_, y_train)
    pickle.dump(gs, open(f'Models/Kingdom/KNN_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/KNN_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### Logistic Regression

In [ ]:
grid = [{'penalty': ['l1'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['saga']},
        {'penalty': ['elasticnet'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'l1_ratio': [0.25, 0.5, 0.75],
         'solver': ['saga']},
        {'penalty': ['l2'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['sag', 'saga']},
       {'penalty': ['none'],
        'fit_intercept': [True, False],
        'class_weight': [None, 'balanced'],
        'multi_class':['ovr', 'multinomial'],
        'solver': ['sag', 'saga']}]
#sag and saga are the fastest for long datasets

In [ ]:
for f_sel in ['all', 'sel']:
    if f_sel == 'sel':
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Kingdom']['Chemical entities']['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train_scal[:, f_index]
    else:
        X_train_ = X_train_scal
    estimator = LogisticRegression(max_iter=5000, tol=1e-6, random_state=seed, n_jobs=-1)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                      error_score='raise', return_train_score=True, n_jobs=-1)
    gs.fit(X_train_, y_train)
    pickle.dump(gs, open(f'Models/Kingdom/LR_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/LR_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/LR_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### SVM

In [ ]:
grid = [{'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'class_weight': [None, 'balanced']}]
#max iter set a big enough
#penalty = l1 and loss = hinge is not supported and l2 and hinge as well when dual=False (which is the case)
#dual=False because n_samples>n_features sklearn
#multi_class "crammer_singer" is comput expensive and usually leads to worse results

In [ ]:
#It was fast, increased max_iter, they were reached in some fits
for f_sel in ['all', 'sel']:
    if f_sel == 'sel':
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Kingdom']['Chemical entities']['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train_scal[:, f_index]
    else:
        X_train_ = X_train_scal
    estimator = LinearSVC(dual=False, loss='squared_hinge', multi_class='ovr', tol=1e-6, max_iter=5000, 
                           random_state=seed, verbose=3)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                      error_score='raise', return_train_score=True, n_jobs=-1)
    gs.fit(X_train_, y_train)
    pickle.dump(gs, open(f'Models/Kingdom/SVM_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/SVM_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### ComplementNB

In [ ]:
grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
        'norm': [True, False]}

In [ ]:
for f_sel in['all', 'sel']:
    if f_sel == 'sel':
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Kingdom']['Chemical entities']['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train_scal[:, f_index]
    else:
        X_train_ = X_train_scal
    estimator = ComplementNB()
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                      error_score='raise', return_train_score=True, n_jobs=-1)
    gs.fit(X_train_, y_train)
    pickle.dump(gs, open(f'Models/Kingdom/NB_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Kingdom/NB_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### Load GS scores

In [ ]:
#Select if result is better with or without f selection
scores_df = [{'Feature selection': 'No', 'F1-score average': 'macro'}, 
            {'Feature selection': 'No', 'F1-score average': 'micro'},
            {'Feature selection': 'Yes', 'F1-score average': 'macro'}, 
            {'Feature selection': 'Yes', 'F1-score average': 'micro'}]
params_df = [{'Classifier':'Chemical entities'}]
for alg in ['RF', 'KNN', 'LR', 'SVM', 'NB']:
    f_sel_gs = {}
    for f_sel in ['all', 'sel']:
        f_sel_gs[f_sel] = pickle.load(open(f'Models/Kingdom/{alg}_{f_sel}_feat.pkl', 'rb'))
    score_all = f_sel_gs['all'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['all'].best_index_]
    score_sel = f_sel_gs['sel'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['sel'].best_index_]
    if score_all >= score_sel:
        f_sel = 'all'
        f = 'No'
    else:
        f_sel = 'sel'
        f = 'Yes'
    gs = f_sel_gs[f_sel]
    #Save scores
    for z, f_sel in enumerate(['all', 'sel']):
        z=z*2
        scores_df[0 + z][f'{alg} (val)'] = f_sel_gs[f_sel].cv_results_[f'mean_test_f1_macro'][f_sel_gs[f_sel].best_index_]
        scores_df[0 + z][f'{alg} (train)'] = f_sel_gs[f_sel].cv_results_[f'mean_train_f1_macro'][f_sel_gs[f_sel].best_index_]
        scores_df[1 + z][f'{alg} (val)'] = f_sel_gs[f_sel].cv_results_[f'mean_test_f1_micro'][f_sel_gs[f_sel].best_index_]
        scores_df[1 + z][f'{alg} (train)'] = f_sel_gs[f_sel].cv_results_[f'mean_train_f1_micro'][f_sel_gs[f_sel].best_index_]
    #Save parameters
    params = gs.cv_results_['params'][gs.best_index_]
    params_df[0][f'{alg} (Feature Selection)'] = f
    for param_name, param in params.items():
        if param == 'squared_hinge':
            continue
        params_df[0][f'{alg} ({param_name})'] = param
scores_df = pd.DataFrame(scores_df).set_index(['Feature selection', 'F1-score average'])
params_df = pd.DataFrame(params_df)

In [ ]:
with pd.ExcelWriter('Results/GS_scores.xlsx', mode='w') as writer:  
    scores_df.to_excel(writer, sheet_name='Kingdom')
scores_df

In [ ]:
params_df['Level'] = 'Kingdom'
params_df.to_pickle('Results/GS_best_comb_params.pkl')

In [ ]:
params_df.to_pickle('Results/GS_best_comb_params.pkl')